In [ ]:
from phonecodes import phonecodes
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from torch.nn import Transformer
from torch import Tensor
from sklearn.model_selection import train_test_split
import copy
import tqdm
import librosa
import seaborn as sns
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import textgrid
from scipy.spatial.distance import euclidean
import plotly.graph_objects as go
import pandas as pd
import jiwer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict
from transformers import AutoProcessor, AutoModelForCTC
from phonemizer.backend.espeak.wrapper import EspeakWrapper
import soundfile as sf
import pickle
_ESPEAK_LIBRARY = r"C:\Program Files\eSpeak NG\libespeak-ng.dll"
EspeakWrapper.set_library(_ESPEAK_LIBRARY)
'''processor_P = AutoProcessor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model_P = AutoModelForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")'''

# loading Hubert model
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
processor_H = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model_H = AutoModelForCTC.from_pretrained("facebook/hubert-large-ls960-ft")

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


c:\Users\Alex\anaconda3\envs\BayesPCN\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForCTC 

## Step 1: Create 3d t-SNE latent speech representation from Hubert CNN last layer

Loading xls file dataframe from xie2021. We only use the experiment 1a

In [5]:
human_result_path=r"..\data\test.xlsx"
human_result = pd.read_excel(human_result_path)
human_result_1a=human_result[human_result["Experiment"]=="1a"]

Load the audio data.  
note: we use different set for exposure and test, they all from ALLSTAR Hint1 set. Each set got 16 sentences.

In [6]:
def get_pathset(paths):
    return [os.path.join(dir, each_file) for dir, mid, files in os.walk(paths) for each_file in files if each_file.endswith(".wav")]
audio_dir =r"..\data\speech_files"
set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]

Generate latent space representation with Hubert-CNN last layer output. Here we generate one sentence's representation for each time. Since too short or too long audio data, such as at the word-level or entire paragraphs, may lead to different transcription mappings.

In [ ]:
from sklearn.manifold import TSNE
def build_tSNE_matrices(audio_dir, set_list, model, processor):
    audio_path=get_pathset(audio_dir)[::-1]
    sentences=[[] for i in range(32)]
    
    #get each talker's audio data
    for each_path in audio_path:
        audio, sr = librosa.load(each_path)
        wave_res = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        tg = textgrid.TextGrid.fromFile(each_path[:-3]+"TextGrid")
        tg_sentence = tg[0]
        for _,i in enumerate(tg[0]):
            if i.mark!="":
                tg_sentence[_-1].maxTime=tg_sentence[_].minTime
        tg_sentence = [i for i in tg_sentence if i.mark!=""]
        tg_sentence=[tg_sentence[i] for i in set_list]
        
        # feed each sentence's audio data into Hubert
        for _, each_sentence in enumerate(tg_sentence):
            start_sentence = int(each_sentence.minTime*16000)
            end_sentence = int(each_sentence.maxTime*16000)
            input=processor(wave_res[start_sentence:end_sentence], sampling_rate=16000, return_tensors="pt").input_values.to(device)
            model.to(device)
            with torch.no_grad():
                out_encoder=model.hubert.feature_extractor(input).transpose(2,1).cpu().numpy()
            sentences[_].append(out_encoder) # store by sentence level.
    
    # flatten data then apply 3d t-SNE.
    flatten=np.array([x for i in sentences for j in i for x in j[0]])
    tsne = TSNE(n_components=3, random_state=42)
    reduced_data = tsne.fit_transform(flatten)
    mean = np.mean(reduced_data, axis=0)
    std = np.std(reduced_data, axis=0)
    reduced_data= (reduced_data - mean) / std
    count=0
    
    for _,i in enumerate(sentences):
        for __,j in enumerate(i):
            sentences[_][__]=reduced_data[count:count+j.shape[1]]
            count+=j.shape[1]
    return sentences

set_list=set1_list+set2_list
sentence_matrix = build_tSNE_matrices(audio_dir, set_list, model_H, processor_H)

In [ ]:
# store the 3d t-SNE data for later usage.


#with open("..\data\hubert_sentences_CNN.pkl", "wb") as file:
#    pickle.dump(sentence_matrix, file)
    


In [ ]:
#load the 3d t-SNE data

#with open("..\data\hubert_sentences.pkl", "rb") as file:
#    sentence_matrix1 = pickle.load(file)

## Step 2: Apply DTW on word-level to calculate the distance.

In [ ]:
def get_keywords_dict(human_result_1a):
    '''
    Return a dict, key: sentenceID, values: keywords
    '''
    keywords_dict={}
    for each_ in human_result_1a.values:
        sentenceID=each_[human_result_1a.columns.get_loc("SentenceID")]
        if sentenceID not in keywords_dict:
            keywords_dict[sentenceID]=[]
        keyword=each_[human_result_1a.columns.get_loc("Keyword")]
        if keyword not in keywords_dict[sentenceID]:
            keywords_dict[sentenceID].append(keyword)
    return dict(sorted(keywords_dict.items()))


In [ ]:
#create a word level 3d feature set.

def create_set(audio_dir, df, reduced_data):
    set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
    set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]
    keywords_dict=get_keywords_dict(df)
    keywords=[j for i in list(keywords_dict.values()) for j in i]
    audio_path=get_pathset(audio_dir)[::-1]
    out_dict={}
    
    word_features=[[] for i in range(len(keywords))]
    for __, each_path in enumerate(audio_path):
        
        current_talker=os.path.basename(each_path)[:13]
        if current_talker not in out_dict.keys():
            out_dict[current_talker]=[[] for i in range(32)]
        tg = textgrid.TextGrid.fromFile(each_path[:-3]+"TextGrid")
        tg_sentence = tg[0]
        for _,i in enumerate(tg[0]):
            if i.mark!="":
                tg_sentence[_-1].maxTime=tg_sentence[_].minTime
        tg_sentence = [i for i in tg_sentence if i.mark!=""]
        tg_sentence=[tg_sentence[i] for i in set1_list+set2_list]
        tg_word = [i for i in tg[1] if i.mark!="" and i.mark!="sp"]
        
        count=0
        for _,each_sentence in enumerate(tg_sentence):
            sentence_total_length=each_sentence.maxTime-each_sentence.minTime
            
            for key_word in list(keywords_dict.values())[_]:
                
                for each_word_tg in tg_word:
                    if each_word_tg.mark.lower()==key_word:
                        if each_word_tg.minTime >= each_sentence.minTime and each_word_tg.maxTime <= each_sentence.maxTime:                        
                            start=each_word_tg.minTime
                            end=each_word_tg.maxTime
                            break

                word_cut_start=start-each_sentence.minTime
                word_cut_end=end-each_sentence.minTime
                word_start=round(reduced_data[_][__].shape[0]*word_cut_start/sentence_total_length)
                word_end=round(reduced_data[_][__].shape[0]*word_cut_end/sentence_total_length)
                
                features=copy.deepcopy(reduced_data[_][__][word_start:word_end,:])
                word_features[count].append(features)
                out_dict[current_talker][_].append(features)
                count+=1

    return word_features,out_dict


word_features,out_dict=create_set(audio_dir, human_result_1a, sentence_matrix)

In [ ]:
def get_training_paths(TrainingTalkerID):
    path_list=[]
    TalkerID=[]
    for each_ID in TrainingTalkerID.split(", "):
        if each_ID[:3]=="CMN":
            TalkerID.append(f"ALL_{each_ID[-3:]}_M_CMN")
        else:
            TalkerID.append(f"ALL_{each_ID[-3:]}_M_ENG")
    return TalkerID

def get_keywords_list(df):
    out_dict={}
    for each_ in human_result_1a.values:
        keyword_loc=df.columns.get_loc("Keyword")
        key_word = each_[keyword_loc]
        sentenceID = each_[df.columns.get_loc("SentenceID")]
        if sentenceID not in out_dict.keys():
            out_dict[sentenceID]=[]
        if key_word not in out_dict[sentenceID]:
            out_dict[sentenceID].append(key_word)
    return out_dict

def get_exposure_set(feature_dict,trainingTalkerID,sentenceID,key_word):
    keywors_list = get_keywords_list(human_result_1a)
    set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
    set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]
    features=[]
    for i in trainingTalkerID:
        sentence_ind=(set1_list+set2_list).index(int(sentenceID[-3:])-1)
        key_word_ind=keywors_list[sentenceID].index(key_word)
        features.append(copy.deepcopy(feature_dict[i][sentence_ind][key_word_ind]))
    return features

def get_test_feature(feature_dict, test_talker, sentenceID, key_word):
    keywors_list = get_keywords_list(human_result_1a)
    set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
    set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]
    #print(sentenceID)
    sentence_ind=(set1_list+set2_list).index(int(sentenceID[-3:])-1)
    key_word_ind=keywors_list[sentenceID].index(key_word)
    
    return copy.deepcopy(feature_dict[test_talker][sentence_ind][key_word_ind])


def sim_measure1(df, feature_dict):
    sim_min_list=[]
    sim_std_list=[]
    sim_median_list=[]
    sim_mean_list=[]
    
    train_set_dict={}
    test_word_dict={}

    train_talker_list=[]
    test_talker_list=[]
    out_df=pd.DataFrame(columns=['Condition2', 'TrainingTalkerID', 'TestTalkerID','SentenceID', 'Keyword',  'distance_min', 'IsCorrect'])#'trial',
    for each_ in tqdm.tqdm(df.values):
        filename_loc=df.columns.get_loc("Filename")
        keyword_loc=df.columns.get_loc("Keyword")
        sentence_loc= df.columns.get_loc("SentenceID")
        training_talker_loc=df.columns.get_loc("TrainingTalkerID")
        
        set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
        set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]
        if each_[df.columns.get_loc("TrainingTestSet")] == "set2,set1":
            train_set=set2_list
            test_set=set1_list
        else:
            train_set=set1_list
            test_set=set2_list
        
        test_file = [os.path.basename(each_[df.columns.get_loc("Filename")])[:13]]
        key_word = each_[keyword_loc] #string
        TrainingTalkerID = each_[training_talker_loc] #list of string
        sentenceID = each_[df.columns.get_loc("SentenceID")]
        trainingTalkerID=get_training_paths(TrainingTalkerID)
        
        train_talker_key=",".join(sorted(trainingTalkerID))
        if train_talker_key not in train_set_dict:
            train_set_dict[train_talker_key]={}
        if sentenceID not in train_set_dict[train_talker_key]:
            train_set_dict[train_talker_key][sentenceID]={}
        if key_word not in train_set_dict[train_talker_key][sentenceID]:
            
            training_features=get_exposure_set(feature_dict,trainingTalkerID,sentenceID,key_word)
            train_set_dict[train_talker_key][sentenceID][key_word]=copy.deepcopy(training_features)
        else:
            training_features=train_set_dict[train_talker_key][sentenceID][key_word]
            
        #training_features=get_exposure_set(feature_dict,trainingTalkerID,sentenceID,key_word)
        
        
        if test_file[0] not in test_word_dict:
            test_word_dict[test_file[0]]={}
        if sentenceID not in test_word_dict[test_file[0]]:
            test_word_dict[test_file[0]][sentenceID]={}
        if key_word not in test_word_dict[test_file[0]][sentenceID]:
            test_feature = get_test_feature(feature_dict, test_file[0], sentenceID, key_word)
            test_word_dict[test_file[0]][sentenceID][key_word]=copy.deepcopy(test_feature)
        else:
            test_feature = test_word_dict[test_file[0]][sentenceID][key_word]
        
        train_talker_list.append(",".join(trainingTalkerID))
        test_talker_list.append(test_file[0])
        sims=[]
        
        for _, each_train_feature in enumerate(training_features):
            #print(each_train_feature)
            #print(test_feature)
            X=each_train_feature.transpose()
            Y=test_feature.transpose()
            D, wp = librosa.sequence.dtw(X, Y, metric='euclidean')
            
            
            
            combined_length = X.shape[1] + Y.shape[1]
            normalized_distance = D[-1, -1] / combined_length

            sims.append(normalized_distance)
            
            #out_df=pd.DataFrame(columns=['Condition2', 'TrainingTalkerID', 'TestTalkerID', 'Keyword','distance_min', 'IsCorrect'])
        out_df.loc[len(out_df)]=[each_[df.columns.get_loc("Condition2")], each_[df.columns.get_loc("TrainingTalkerID")],#trainingTalkerID[_],
                                    each_[df.columns.get_loc("TestTalkerID")], each_[sentence_loc],
                                    each_[df.columns.get_loc("Keyword")], 
                                    #each_[df.columns.get_loc('trial')],
                                    sims, 
                                    each_[df.columns.get_loc("IsCorrect")]]

        
        #sim_min=np.min(sims)
        #sim_std=np.std(sims)
        #sim_median=np.median(sims)
        #sim_mean=np.mean(sims)
        #
        #sim_min_list.append(sim_min)
        #sim_std_list.append(sim_std)
        #sim_median_list.append(sim_median)
        #sim_mean_list.append(sim_mean)

        
    return out_df



out_df=sim_measure1(human_result_1a, out_dict)
new_df=copy.deepcopy(out_df)
#convert distance into similarity
new_df['similarity'] = np.exp(-1*new_df['distance_min'].apply(lambda x:np.min(x)))

#groupby 3296 unique combination
TTID=new_df.columns.get_loc("TrainingTalkerID")
new_TrainingTalkerID=[]
for i in new_df.values:
    new_TrainingTalkerID.append( ",".join(sorted(i[TTID].split(", "))) )
new_df["TrainingTalkerID1"]=new_TrainingTalkerID
new_df['trial'] = (new_df.groupby(['Keyword', 'Condition2', 'TrainingTalkerID1', 'TestTalkerID', 'SentenceID'], as_index=False).ngroup() + 1)


new_df2=new_df.groupby(['Keyword', 'Condition2', 'TrainingTalkerID1', 'TestTalkerID', 'SentenceID','trial'], as_index=False).agg( #'sentenceID'
    IsCorrect=('IsCorrect', 'mean'),
    #distance=('distance_min', 'min'),
    similarity=('similarity', 'mean'),
    numCorrect=('IsCorrect', lambda x: (x==1).sum()),
    numIncorrect=('IsCorrect', lambda x: (x==0).sum())
    )
new_df2.to_csv('..\data\df_CNN_k=1_max.csv', index=False)
#new_df2.to_csv('new_df2m.csv', index=False)
